In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import json
import os
import sympy as sp

# Set up plotting style
plt.style.use('seaborn-v0_8-darkgrid')
%matplotlib inline

In [3]:
q1, q2, q3 = sp.symbols('q1 q2 q3', real=True)
dq1, dq2, dq3 = sp.symbols('dq1 dq2 dq3', real=True)
ddq1, ddq2, ddq3 = sp.symbols('ddq1 ddq2 ddq3', real=True)

q  = sp.Matrix([q1, q2, q3])
dq = sp.Matrix([dq1, dq2, dq3])
ddq= sp.Matrix([ddq1, ddq2, ddq3])

m1, m2, m3 = sp.symbols('m1 m2 m3', positive=True)
l1, l2, l3 = sp.symbols('l1 l2 l3', positive=True)
a1, a2, a3 = sp.symbols('a1 a2 a3', real=True)  # положение CoM вдоль звена
I1, I2, I3 = sp.symbols('I1 I2 I3', positive=True)
g = sp.symbols('g', positive=True)

In [4]:
# --- helper ---
def rot(th):
  return sp.Matrix([sp.cos(th), sp.sin(th)])

In [5]:
th1 = q1
th2 = q1 + q2
th3 = q1 + q2 - q3

r1 = a1 * rot(th1)
r2 = l1*rot(th1) + a2*rot(th2)
r3 = l1*rot(th1) + l2*rot(th2) + a3*rot(th3)

In [6]:
J1 = r1.jacobian(q)
J2 = r2.jacobian(q)
J3 = r3.jacobian(q)

v1 = J1*dq
v2 = J2*dq
v3 = J3*dq

w1 = sp.diff(th1, q1)*dq1 + sp.diff(th1, q2)*dq2 + sp.diff(th1, q3)*dq3
w2 = sp.diff(th2, q1)*dq1 + sp.diff(th2, q2)*dq2 + sp.diff(th2, q3)*dq3
w3 = sp.diff(th3, q1)*dq1 + sp.diff(th3, q2)*dq2 + sp.diff(th3, q3)*dq3

In [7]:
T = sp.Rational(1,2)*(m1*(v1.dot(v1)) + I1*w1**2 +
                      m2*(v2.dot(v2)) + I2*w2**2 +
                      m3*(v3.dot(v3)) + I3*w3**2)

P = m1*g*r1[1] + m2*g*r2[1] + m3*g*r3[1]

In [8]:
dT_ddq = sp.Matrix([sp.diff(T, dqi) for dqi in dq])  # ∂T/∂dq

d_dt_dT_ddq = dT_ddq.jacobian(q)*dq + dT_ddq.jacobian(dq)*ddq

dT_dq = sp.Matrix([sp.diff(T, qi) for qi in q])
dP_dq = sp.Matrix([sp.diff(P, qi) for qi in q])

tau = sp.simplify(d_dt_dT_ddq - dT_dq + dP_dq)

In [9]:
M = tau.jacobian(ddq)          # коэффициенты при ddq
h = sp.simplify(tau - M*ddq)   # всё остальное

# Если нужно отдельно:
zero_dq  = {dq1:0, dq2:0, dq3:0}
zero_ddq = {ddq1:0, ddq2:0, ddq3:0}

G   = sp.trigsimp(h.subs(zero_dq))   # h(q,0)
Cdq = sp.trigsimp(h - G)             # C(q,dq)*dq

print("M(q) ="); sp.pprint(sp.trigsimp(M))
print("\nG(q) ="); sp.pprint(G)
print("\nC(q,dq)*dq ="); sp.pprint(Cdq)

M(q) =
⎡                 2         ⎛  2                       2⎞      ⎛  2            ↪
⎢I₁ + I₂ + I₃ + a₁ ⋅m₁ + m₂⋅⎝a₂  + 2⋅a₂⋅l₁⋅cos(q₂) + l₁ ⎠ + m₃⋅⎝a₃  + 2⋅a₃⋅l₁⋅ ↪
⎢                                                                              ↪
⎢                      2                           2                           ↪
⎢          I₂ + I₃ + a₂ ⋅m₂ + a₂⋅l₁⋅m₂⋅cos(q₂) + a₃ ⋅m₃ + a₃⋅l₁⋅m₃⋅cos(q₂ - q₃ ↪
⎢                                                                              ↪
⎢                                                 2                            ↪
⎣                                         -I₃ - a₃ ⋅m₃ - a₃⋅l₁⋅m₃⋅cos(q₂ - q₃) ↪

↪                                    2                       2⎞                ↪
↪ cos(q₂ - q₃) + 2⋅a₃⋅l₂⋅cos(q₃) + l₁  + 2⋅l₁⋅l₂⋅cos(q₂) + l₂ ⎠  I₂ + I₃ + a₂⋅ ↪
↪                                                                              ↪
↪                                               2                              ↪
↪ ) + 2⋅a₃⋅l₂⋅m₃⋅cos